In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import pycountry

import pdfplumber


In [3]:
os.getcwd()
#os.chdir("/Users/veesheen/Desktop")

'/Users/veesheenyuen/code/veeyuen/SAA'

In [4]:
countries=list(pycountry.countries)

In [5]:
countries[0].alpha_3

'ABW'

In [16]:
# missing KSA, VIE, INA, OMA, QAT, SRI, UAE, TPE, NEP, PLE, KUW, PHI, MAS

list_countries=[]

for i in range(len(countries)):
    
    list_countries.append(countries[i].alpha_3)

In [17]:
list_countries = list_countries + ['IRI', 'KSA', 'VIE', 'INA', 'OMA', 'QAT', 'SRI', 'UAE', 'TPE', 'NEP', 'PLE', 'KUW', 'PHI', 'MAS']

In [69]:
# Scrape U20 2024 PDFs

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/NSG2024/')

#file = "/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6.pdf"

file = "/Users/veesheenyuen/Desktop/DataScience/SAA/U20/All Results.pdf"

 
# Iterate over files in directory

event_pattern ='(MEN U20 -)|(WOMEN U20 -)'

heat_pattern = 'HEAT \d'

school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
year_pattern='\d\d\d\d'

#name_pattern='\d\d\d.{10-29}\d\d\d\d'

stage_pattern='FINAL|QUALIFYING|ROUND|SEMI'

country_code = '\s[A-Z]{3}\s'

DOB_pattern='\s20[0-9]{2}\s'

#tag_pattern='\s[0-9]{3}\s'

tag_pattern='[0-9]{3}\s'


wind_pattern='(\+\d\.\d|\-\d\.\d)'

name_pattern='[a-zA-Z\-\. \"]{3,31}'

results_begin = '^\d'  # line begins with a digit

react_time = '0.[0-9]{3}'




df_table=pd.DataFrame()  # initialize empty master df

df=None

temp=pd.DataFrame(columns = ['Pos.', 'BIB', 'NAME', 'Born', 'Nation', 'Result', 'Wind', 'React', 'Time', 'Points'])



with pdfplumber.open(file) as pdf:
        
        
    for i in range(len(pdf.pages)):    
        
        page = pdf.pages[i]  # can iterate over different pages
        table=page.extract_table()
        text=page.extract_text()        
        
        df=pd.DataFrame(text.splitlines())
        
                        
        for index, row in df.iterrows(): # find row containing event details and column names  
            
                                    
        
            if 'CHAMPIONSHIPS' in row[0]:
            
                event=row[0]
                                                
            if 'MEN' in row[0]:
                gender='Male'
                
            if 'WOMEN' in row[0]:
                gender='Female'    
                
            
            # Extract event and stage   
                        
            if re.search(event_pattern, row[0]): # isolate event in two steps
                                
                epos = re.search(event_pattern, row[0])  # find row
                
                start = epos.end()
                
                string = row[0][start:] # strip out event part
                
                                
                try:
                    spos = re.search(stage_pattern, string)  # check if row contains stage indication
                    split_end = spos.start()-1
                    
                    event = string[:split_end]   # strip out stage
                    
                    stage = string[split_end:]
                                        
                    
                except:
                    
                    event = string.strip()                    
                    
                
                                 
                    
            # Extract Heat
            
            if 'HEAT' in row[0]:
                                
                heat=re.findall(heat_pattern, row[0])
                                                                    
                
            # Extract columns of table    
            
            if 'Pos' in row[0]: # extract column names
                            
                row_index=index  # where the header row is
                columns=df.iloc[row_index,0]
                column_names=columns.split(' ')
                                
                column_names.remove('First')
                column_names.remove('LAST')
                column_names.remove('Name,')            

             
            # Extract alpha 3 country code. Look for it after DOB
            
            
            try:
                                            
                find = re.search(DOB_pattern, row[0])
            
                start = find.end()
                end = find.end()+3
            
                country = row[0][start:end].strip()
                                            
                if country in list_countries:
                       
                    pass
                
                else:
                    
                    country = None
                
                
            except:
                
                pass
            
            
            # Extract TAG ID
            
            res = re.findall(tag_pattern, row[0])
            
            if len(res)>1:
                            
                tag_id = res[0]
                
                
            else:
                
                tag_id = res
            
            
                        
            # Extract DOB
            
            dob = re.findall(DOB_pattern, row[0])
                        
                                    
            
            # Extract name (will pick up nation code as a 3rd element in list)
                        
          #  if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                
            
            if re.search(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                                  
                result = re.findall(name_pattern, row[0])   # does not catch some names corrupted at the end
                                              
                full_name = result[0]  
                
                        
              #  list=row[0].split(' ')
                                
              #  full_name = list[2]  
                                
                
                                                            
              
            # Extract reaction time
            
            if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                            
            
                reaction = re.findall(react_time, row[0])
                
                            
                
            # Extract wind
            
            if re.findall(DOB_pattern, row[0]) and re.search(results_begin, row[0]):
                            
            
                wind = re.findall(wind_pattern, row[0])
                
                        
                

            # Extract result. Put into a list, search for country code index and choose index after
            # Look for existance of tag_id, dob to find valid line
            
            
            if dob and tag_id:  # lines with DQ and DNS are not picking up because tag_id for those has no space in front
                
                
                list = row[0].split(' ')
                                
                
                try:
                
                    index = list.index(country)
                                        

                    result = list[index+1]
                                        
                                                            
                except:
                    
                    pass

                
                
    #            position1 = re.findall(country_code, row[0])
    #            
    #            if len(position1)>1:  # choose second match if more than one
    #            
    #                pos1=position1[1]
                                        
    #                start=re.search(pos1, row[0])
                                        
     #           else:
                    
    #                start=re.search(position1[0], row[0])
                    
                                        
                            
                            
    #            begin=start.end()
    #            end=start.end()+8
                
    #            result = row[0][begin:end]
                
    #            print(result)
                            
                
            # Extract points. If 'points' is present in list of columns extract it from last element in list
                        
            if dob and tag_id and 'Points' in column_names and 'DNS' not in row[0]:
                
                
                list=row[0].split(' ')
                                
                points = list[-1]  
                
                rank = list[0]
                                
            if dob and tag_id and 'Pos.' in column_names:
                
            
                list=row[0].split(' ')
                                
                rank = list[0]  
                                             
                
            #    split_start = position.end()
            
           #     remainder = row[0][split_start:]
                
           #     into_list=remainder.split(' ')
                
           #    result=into_list[0]
                
             #   print(result)
                
                
             #   print('here',remainder)
            
            

                
                                       
            if 'W/Gauge' in row[0]:
                
                details=row[0]            
                
                try:
            
                    wind=re.findall(wind_pattern, details)
                                    
                    wind_dict = {}
                    
                    for key, val in wind: # create dictionary of extracted wind data
                        wind_dict.setdefault(key, val)
                
                except:
                    
                    pass
                
          #  temp.at[i, 'Born'] = dob[0]
        
   #         print(tag_id, full_name, dob, country, wind, reaction, result, points)
        
            if event == ' 400m'or event == '400m':
            
                if tag_id and dob:
            
                
        
                    print('here', event, rank, tag_id, full_name, dob, country, wind, reaction, result, points, heat, stage, gender)

            
        
    
    
                    
        temp=pd.DataFrame(table)
                
        points = None
        reaction = None
        wind = None
        event = None
        country = None
        dob = None
        full_name = None
        tag_id = None
        rank = None
        result = None
        
        
        
                                    
                
 #       temp.iloc[2:,:]  # drop first 2 rows

 #       temp['EVENT'] = final_match
 #       temp['GENDER'] = gender
 #       temp['DIVISION'] = div
 #       temp['STAGE'] = stage
 #       temp['COMPETITION'] = 'NSG'
 #       temp['REGION'] = 'Local'
 #       temp['CATEGORY_EVENT'] = category_event
 #       temp['test'] = 'H'+temp[5].map(str) # create a new temp field to map across wind
 #       temp['SOURCE'] = 'PDF'
        
 #       temp['DATE'] = '2024'
 #       temp['QUALIFICATION']=''
 #       temp['POINTS']=''
 #       temp['UNIQUE_ID']=''
 #       temp['COUNTRY']=''
 #       temp['DICT_RESULTS']=''
 #       temp['DOB']=''
 #       temp['GROUP']=''
 #       temp['ATHLETE_ID']=''
 #       temp['TIMESTAMP']=''
 #       temp['FREE_FIELD']=''
 #       temp['FREE_FIELD2']=''
 #       temp['FREE_FIELD3']=''

        
  #      temp['WIND']=temp['test'].map(wind_dict) # map temp field across to wind data
        
  #      temp = temp[temp[0].str.contains("~~~ End of Listing ~~~") == False]  # drop useless rows
  #      temp = temp[temp[0].str.contains("W/Gauge :") == False]
  #      temp = temp[temp[0].str.contains("Pos") == False]
     #   temp = temp[temp[7].str.contains("DNS|NM|DQ|DNF") == False] # drop rows without results
        
            
        df_table=pd.concat([df_table, temp], axis=0)
         
        


#df_table.rename(
#    columns={0: "RANK", 1: "Tag", 2: "NAME", 3: "TEAM", 4: "RESULT", 5: "HEAT", 6: "LANE", 7: "REMARKS"},
#    inplace=True,
#)
        
#df_table.drop('test', axis=1, inplace=True)    # drop temp columns used to map wind data   

#df_table = df_table.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
#                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
#                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'])


#df_table.to_csv('sectrack_result_22a.csv', sep=',', index=False, encoding='utf-8')
        

            
                       
                       

        
            
                


here 400m 1 603   Hukmova JONBIBI  [' 2005 '] UZB [] ['0.173'] 55.23 None ['HEAT 1']  ROUND 1 Female
here 400m 2 414   Yalun WANG  [' 2005 '] CHN [] ['0.416'] 56.30 None ['HEAT 1']  ROUND 1 Female
here 400m 3 513   Kidenko SOFYA  [' 2007 '] KAZ [] ['0.151'] 56.39 None ['HEAT 1']  ROUND 1 Female
here 400m 5 576   Angkana THONGTAE  [' 2006 '] THA [] ['0.364'] 59.06 None ['HEAT 1']  ROUND 1 Female
here 400m 1 524   Shuvalova MARIYA  [' 2006 '] KAZ [] ['0.193'] 56.03 None ['HEAT 2']  ROUND 1 Female
here 400m 2 465   Sai Sangeetha DODLA  [' 2006 '] IND [] ['0.302'] 56.21 None ['HEAT 2']  ROUND 1 Female
here 400m 3 565   Laavinia JAIGANTH  [' 2006 '] SGP [] ['0.352'] 59.40 None ['HEAT 2']  ROUND 1 Female
here 400m 4 548   Hawwa Muznaa FAIZ  [' 2006 '] MDV [] ['0.393'] 59.77 None ['HEAT 2']  ROUND 1 Female
here 400m 5 542   Joy El ESTA  [' 2007 '] LBN [] ['0.307'] 01:00.89 None ['HEAT 2']  ROUND 1 Female
here 400m 595 595   Mariam KAREEM  [' 2008 '] UAE [] [] DNS None ['HEAT 2']  ROUND 1 Fema

In [340]:
import pdfplumber
import os


os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/U20/') 

#file="sectrack_audit_2010_211-211.pdf"
#file="Session 13_13-end.pdf"
#file="Session 13_1-4.pdf"
#file="sectrack_audit_2010_28-28.pdf"
#file="Session 13_5-6.pdf"
file='All Results.pdf'   # stranded pdf



#table_settings = {
#    "vertical_strategy": "text",
#    "horizontal_strategy": "text",
#    }


with pdfplumber.open(file) as pdf:
    
    page = pdf.pages[6]
    table=page.extract_table()
    text=page.extract_text()


splitted=text.splitlines()

splitted



['21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS',
 'DUBAI, 24TH TO 27TH APRIL 2024',
 'Result Published at: 24/04/2024 , 9:30 AM',
 'WOMEN U20 -100m ROUND 1',
 'HEAT 1',
 'Pos. BIB First Name, LAST NAME Born Nation Result Wind React Time',
 '1 492 Morimoto MISAKI 2005 JPN 12.20 Q -1.4m/s 0.168',
 '2 412 Nuoxin CHEN 2008 CHN 12.35 Q -1.4m/s 0.257',
 '3 425 Cheuk Man Serena TANG 2005 HKG 12.41 q -1.4m/s 0.163',
 '4 546 Izzatul Musfirah Binti Ahmad Kama2l 0A0Z8IRA MAS 12.73 -1.4m/s 0.236',
 '5 444 Shava Salvia WARANGGANI 2008 INA 12.79 -1.4m/s 0.180',
 'HEAT 2',
 'Pos. BIB First Name, LAST NAME Born Nation Result Wind React Time',
 '1 419 Yujie CHEN 2008 CHN 11.76 Q -0.5m/s 0.179',
 '2 613 Tran THỊ 2005 VIE 11.78 Q -0.5m/s 0.181',
 '3 579 Jirapat KHANONTA 2005 THA 12.35 q -0.5m/s 0.253',
 '4 547 Ormilla Octavia Codday NYADAU 2006 MAS 12.46 -0.5m/s 0.220',
 '5 526 Yermola KRISTINA 2005 KAZ 12.48 -0.5m/s 0.201',
 '6 604 Karina KARIMOVA 2008 UZB 12.85 -0.5m/s 0.212',
 'HEAT 3',
 'Pos. BIB First Nam

In [341]:
table

[['HEAT 2'],
 ['Pos. BIB First Name, LAST NAME Born Nation Result Wind React Time'],
 ['1 419 Yujie CHEN 2008 CHN 11.76 Q -0.5m/s 0.179'],
 ['2 613 Tran THỊ 2005 VIE 11.78 Q -0.5m/s 0.181'],
 ['3 579 Jirapat KHANONTA 2005 THA 12.35 q -0.5m/s 0.253'],
 ['4 547 Ormilla Octavia Codday NYADAU 2006 MAS 12.46 -0.5m/s 0.220'],
 ['5 526 Yermola KRISTINA 2005 KAZ 12.48 -0.5m/s 0.201'],
 ['6 604 Karina KARIMOVA 2008 UZB 12.85 -0.5m/s 0.212'],
 [''],
 ['']]

In [25]:
pd.DataFrame(splitted)

,0
0,21ST ASIAN U20 ATHLETICS CHAMPIONSHIPS
1,"DUBAI, 24TH TO 27TH APRIL 2024"
2,"Result Published at: 22/04/2024 , 7:06 PM"
3,WOMEN U20 - High Jump FINAL
4,"Pos. BIB First Name, LAST NAME Born Nation Result"
5,1 598 Barnokhon SAYFULLAEVA 2005 UZB 1.83
6,2 521 Meirmanova FAINA 2006 KAZ 1.76 PB
7,3 601 Emiliya RUDINA 2006 UZB 1.74 PB
8,4 518 Malyugina ARINA 2007 KAZ 1.74
9,5 439 Zhichu YANG 2007 HKG 1.65


In [382]:
event_pattern='(MEN U20 -)|(WOMEN U20 -)'
stage_pattern='FINAL|QUALIFYING ROUND|ROUND 1|SEMI FINAL'
#name_pattern='[a-zA-Z]{3,20}'
#name_pattern='[A-Z]{1}[a-z]{2,10}'
#name_pattern='\b[A-Z][a-z]{2-10}[A-Z]\b'
mixed_up = '[a-zA-Z0-9]{5,}'

country_code = '\s[A-Z]{3}\s'
DOB_pattern='20[0-9]{2}\s'
tag_pattern='\s[0-9]{3}\s'


#text = 'WOMEN U20 - High Jump FINAL'
#text = '1 429 Hoi Man Chloe PAK 2006 HKG 22.23 Q -1.1m/s 0.156'
# text = '4 546 Izzatul Musfirah Binti Ahmad Kama2l 0A0Z8IRA MAS 12.73 -1.4m/s 0.236'
# text ='4 288 Al Mulham Said Al Abd Obaid AL SH2E0K0I5LI OMA 10.92 +2.0m/s 0.171 PB'
text = '337 Rajapaksha Pathiranage Don Avishka T3h3e4mindaJ aRyAaJtAhPuAnKgSaHlaAge Omel Shashintha SILVA'
# text = '568 Ambepitiya Wijethunga Gamage Jithm5a6 S9henarDi WunIuJEsTuHngUhNeG MAudiyanselage Maheema Pravindi DUNUSINGHE'
# text = '338 Uyangoda Hewage Hasindu NETHSARA333 Gonsalu Wasan Jathya KIRULU'
# text = '339 Warnakulasooriya Malith Koushan THA3M40EL Weeraratna Dineth INDUWARA'
# text = '278 Muhammad Sayyid Amin Bin ROSLAN274 Dexter BEDIE'
# text = '4 332 Edirisingha Siriwardanalage Denet2h0 A0N5UHAS SRI 2.06'
# text = '5 330 Agampodi Tharusha Dinushan MEN20D0IS8 SRI 2.06'
# text = '337 Rajapaksha Pathiranage Don Avishka T5h7e1mindaK RoAdiJtAhPuAwKaSkHkAu Takshima NUHANSA'
# text = '10 290 Hajan Abdullah Yousuf Ahmed AL-H20A0JR6I OMA 56.04 X X 56.04'
# text = '11 371 Nasser Saeed Nasser Saeed ALKATH20E0E6RI UAE 54.21 54.21 50.54 48.34 PB'
# text = '4 572 Mahadura Gedara Tharushi Abhish2e0k0a8 PEMASIRISRI 02:10.52'
# text = '1 314 Seifeldin Mohamed A ABDELSALAM2005 QAT 5.51'
# text = '6 335 Kuda Liyanage Ayomal AKALANKA2006 SRI 52.84 0.251'
# text = '1 481 Nazanin Fatemeh EIDIYANKAKHAK2I005 IRI 58.86 0.239 PB'
# text = '8 373 Salem Yousufsalem Mohammad Al2 M00E8QBAALI UAE 12.76 12.73 X 12.76 X X 12.35'
# text ='7 367 Hasan Taleb Hasan Ahmed ALDHUH20O0O7RI UAE 11.43 -0.8m/s 0.182'
# text = '5 309 Mohammed Ghanim M A AL-KUWA20R0I7 QAT 11.03 +1.0m/s 0.140 PB'
# text = '5 309 Mohammed Ghanim M A AL-KUWA20R0I7 QAT 11.03 +1.0m/s 0.140 PB'
# text = '3 171 Seyed Mohammad Sajjad AGHAEI2005 IRI 47.99 q 0.241'
text = '4 431 Jane Christa Ming Suet KARLSSON2007 HKG 56.98 q 0.139'
#text = '1 314 Seifeldin Mohamed A ABDELSALAM2005 QAT 5.51'

# text = '10 290 Hajan Abdullah Yousuf Ahmed AL-H20A0JR6I OMA 56.04 X X 56.04'
# text = '11 371 Nasser Saeed Nasser Saeed ALKATH20E0E6RI UAE 54.21 54.21 50.54 48.34 PB'


#text = '5 552 Ala"a Abdulla Mohamed AL-ZADJA2LI005 OMA 22.69 -1.1m/s 0.212'

result = re.findall(mixed_up, text)  


In [383]:
result

['Christa', 'KARLSSON2007']

In [292]:
text_list=text.split(' ')

In [291]:
print(re.sub("[a-z] [A-Z]", ".", text))

1 429 Ho.a.hlo.AK 2006 HKG 22.23 Q -1.1m/s 0.156


In [293]:
idx = [i for i, item in enumerate(text_list) if re.search(name_pattern, item)]


In [294]:
idx

[2, 3, 4]